In [1]:
import sys
import logging

import http.client
import json

import pandas as pd

In [2]:
logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
logger.setLevel(logging.DEBUG)

c_handler = logging.StreamHandler(sys.stdout)
c_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# c_format = logging.Formatter(f'%(levelname)-8s: %(message)s')
c_handler.setFormatter(c_format)

for handler in logger.handlers:
    logger.removeHandler(handler)
logger.addHandler(c_handler)

In [3]:
logger.debug('meow')
logger.info('aku adalah anak woof woof')

2020-03-10 18:18:56,411 - __main__ - DEBUG - meow
2020-03-10 18:18:56,421 - __main__ - INFO - aku adalah anak woof woof


In [4]:
TYPE = 'train'

SQUAD_DATASET_PATH = f'Datasets/SQuAD/v2.0/{TYPE}-v2.0-translated_fixed_enhanced.json'

df_squad = pd.read_json(SQUAD_DATASET_PATH)
df_squad = df_squad
print(df_squad.shape)
print(df_squad)

(442, 2)
                                           title  \
0                                        Beyonce   
1                                Frédéric Chopin   
2    Hubungan Tiongkok-Tibet selama dinasti Ming   
3                                           IPod   
4         The Legend of Zelda: Twilight Princess   
..                                           ...   
437                                      Infeksi   
438                                      Berburu   
439                                    Kathmandu   
440                               Infark miokard   
441                                      Masalah   

                                            paragraphs  
0    [{'qas': [{'question': 'Kapan Beyonce mulai me...  
1    [{'qas': [{'question': 'Bagaimana kewarganegar...  
2    [{'qas': [{'question': 'Siapakah Wang Jiawei d...  
3    [{'qas': [{'question': 'Perusahaan mana yang m...  
4    [{'qas': [{'question': 'Apa kategori game Lege...  
..                      

In [6]:
def get_pos_tag(text):
    conn = http.client.HTTPConnection("10.181.131.244:5500")
    payload = json.dumps({"text": text})
    headers = {
        'content-type': "application/json",
        'x-api-key': ""
        }

    conn.request("POST", "/", payload, headers)
    res = conn.getresponse()
    data = json.loads(res.read().decode("utf-8"))

    return data

def get_ner(text):
    conn = http.client.HTTPConnection("10.181.131.244:10009")
    payload = json.dumps({"text": text})
    headers = {
        'content-type': "application/json",
        'x-api-key': ""
        }

    conn.request("POST", "/", payload, headers)
    res = conn.getresponse()
    data = json.loads(res.read().decode("utf-8"))

    return data

text = "Frédéric Chopin! adalah, (1907–1986) (1907-1986) anak-anak (children): gembala. Andi's sheep is an '03 R&B player; 奉獻 km² Jīdū °5 вера ʰp. Totalnya 10.000 rupiah u?yea u!yea u,yea u.yea kura-kura bolak-balik"
pos_tag = get_pos_tag(text)
ner = get_ner(text)

In [7]:
print(pos_tag)
print(ner)

{'postags': [[['Frederic', 'NNP'], ['Chopin', 'NNP'], ['!', 'PUN']], [['adalah', 'VBL'], [',', 'PUN'], ['(', 'PUN'], ['1907', 'NUM'], ['-', 'PUN'], ['1986', 'NUM'], [')', 'PUN'], ['(', 'PUN'], ['1907', 'NUM'], ['-', 'PUN'], ['1986', 'NUM'], [')', 'PUN'], ['anak-anak', 'NNO'], ['(', 'PUN'], ['children', 'NNO'], [')', 'PUN'], [':', 'PUN'], ['gembala', 'NNO'], ['.', 'PUN']], [['Andi', 'NNP'], ["'", 'PUN'], ['s', 'NNP'], ['sheep', 'NNP'], ['is', 'PPO'], ['an', 'PUN'], ["'", 'PUN'], ['03', 'NUM'], ['R', 'NNP'], ['&', 'PUN'], ['B', 'NNP'], ['player', 'NNP'], [';', 'PUN'], ['Feng', 'NNP'], ['Xian', 'NNP'], ['km2', 'NNP'], ['Jidu', 'NNP'], ['deg5', 'NNP'], ['vera', 'NNP'], ['kp', 'NNP'], ['.', 'PUN']], [['Total', 'NNO'], ['nya', 'PRK'], ['10.000', 'NUM'], ['rupiah', 'NNO'], ['u', 'ADJ'], ['?', 'PUN']], [['yea', 'NUM'], ['u', 'NNP'], ['!', 'PUN']], [['yea', 'NNO'], ['u', 'PUN'], [',', 'PUN'], ['yea', 'NNP'], ['u.ye', 'NNP'], ['a', 'PPO'], ['kura-kura', 'NNO'], ['bolak-balik', 'VBI']]]}
{'entiti

In [8]:
SLEEP_TIME = 10

def save_wait_recall_api(payload, call_api_func, df, path):
    print(f'Problem when calling API.. Saving, then sleeping for {SLEEP_TIME} seconds...')
    df.to_json(path)
    time.sleep(SLEEP_TIME)
    return call_api_func(payload)

In [12]:
class LastIdxTracker():
    def __init__(self, last_topic_idx, last_content_idx):
        self.topic = last_topic_idx
        self.content = last_content_idx

In [18]:
def add_postag_and_ner(taken_topic_idx, taken_context_idx, path, last_idx_tracker, logger=logger):
    qas = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas']
    context = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['context']
#     logger.debug(f'{context}\n')

    try:
        context_postags = get_pos_tag(context)
    except:
        context_postags = save_wait_recall_api(context, get_pos_tag, df_squad, path)
                    
    try:
        context_ner = get_ner(context)
    except:
        context_ner = save_wait_recall_api(context, get_ner, df_squad, path)
        
    df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx].update(context_postags)
    df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx].update(context_ner)
    
    last_idx_tracker.topic = taken_topic_idx
    last_idx_tracker.content = taken_context_idx
    
    for question in qas:
        try:
            question_postags = get_pos_tag(question['question'])
        except:
            question_postags = save_wait_recall_api(question['question'], get_pos_tag, df_squad, path)

        try:
            question_ner = get_ner(question['question'])
        except:
            question_ner = save_wait_recall_api(question['question'], get_ner, df_squad, path)
        question.update(question_postags)
        question.update(question_ner)

In [21]:
last_idx_tracker = LastIdxTracker(0, 0)

In [23]:
print(last_idx_tracker.topic)
print(last_idx_tracker.content)

111
6


In [ ]:
import time


SAVE_PATH = f'Datasets/SQuAD/v2.0/{TYPE}-v2.0-translated_fixed_enhanced.json'

start_time = time.time()
for taken_topic_idx in range(last_idx_tracker.topic, df_squad.shape[0]):
    logger.debug(f'Topic: {taken_topic_idx}')
    for taken_context_idx in range(last_idx_tracker.content, len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        logger.debug(f'\t{taken_context_idx}')
        add_postag_and_ner(taken_topic_idx, taken_context_idx, SAVE_PATH, last_idx_tracker)
#         logger.debug(df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx])
    last_idx_tracker.content = 0
#     if taken_topic_idx > 4: break
print(time.time() - start_time)

2020-03-10 21:58:56,725 - __main__ - DEBUG - Topic: 111
2020-03-10 21:58:56,729 - __main__ - DEBUG - 	6
2020-03-10 21:58:58,735 - __main__ - DEBUG - 	7
2020-03-10 21:59:03,638 - __main__ - DEBUG - 	8
2020-03-10 21:59:09,683 - __main__ - DEBUG - 	9
2020-03-10 21:59:21,784 - __main__ - DEBUG - 	10
2020-03-10 21:59:23,793 - __main__ - DEBUG - 	11
2020-03-10 21:59:25,633 - __main__ - DEBUG - Topic: 112
2020-03-10 21:59:25,635 - __main__ - DEBUG - 	0
2020-03-10 21:59:26,833 - __main__ - DEBUG - 	1
2020-03-10 21:59:28,083 - __main__ - DEBUG - 	2
2020-03-10 21:59:29,024 - __main__ - DEBUG - 	3
2020-03-10 21:59:30,135 - __main__ - DEBUG - 	4
2020-03-10 21:59:30,988 - __main__ - DEBUG - 	5
2020-03-10 21:59:32,393 - __main__ - DEBUG - 	6
2020-03-10 21:59:33,604 - __main__ - DEBUG - 	7
2020-03-10 21:59:34,783 - __main__ - DEBUG - 	8
2020-03-10 21:59:35,813 - __main__ - DEBUG - 	9
2020-03-10 21:59:37,133 - __main__ - DEBUG - Topic: 113
2020-03-10 21:59:37,135 - __main__ - DEBUG - 	0
2020-03-10 21:

In [16]:
df_squad.iloc[taken_topic_idx]['paragraphs'][1]

{'qas': [{'question': 'Berapa banyak terawatt radiasi matahari yang diterima bumi?',
   'id': '56ce59c8aab44d1400b886dc',
   'answers': [{'text': '174.000', 'answer_start': 19}],
   'is_impossible': False,
   'indonesian_answers': [{'text': '174.000',
     'answer_start': 14,
     'answer_end': 21}]},
  {'question': 'Berapa persentase radiasi matahari yang dipantulkan kembali oleh atmosfer?',
   'id': '56ce59c8aab44d1400b886dd',
   'answers': [{'text': '30%', 'answer_start': 122}],
   'is_impossible': False,
   'indonesian_answers': [{'text': '30%',
     'answer_start': 106,
     'answer_end': 109}]},
  {'question': 'Daerah tempat tinggal orang biasanya menerima kisaran kWh / m2 per hari?',
   'id': '56ce59c8aab44d1400b886de',
   'answers': [{'text': '3,5 hingga 7,0', 'answer_start': 476}],
   'is_impossible': False,
   'indonesian_answers': [{'text': '3,5 hingga 7,0 ',
     'answer_start': 475,
     'answer_end': 490}]},
  {'question': 'Berapa banyak radiasi yang diterima bumi?',
   '

In [59]:
df_squad = df_squad[df_squad.index < 9]
df_squad.iloc[-1]['paragraphs'][-1]

{'qas': [{'question': 'Bagaimana warga Monroeville mengutip garis-garis buku?',
   'id': '56d3f1872ccc5a1400d82f75',
   'answers': [{'text': 'seperti Kitab Suci', 'answer_start': 115}],
   'is_impossible': False,
   'indonesian_answers': [{'text': 'seperti Kitab Suci',
     'answer_start': 133,
     'answer_end': 151}],
   'postags': [[['Bagaimana', 'ADV'],
     ['warga', 'NNO'],
     ['Monroeville', 'NNP'],
     ['mengutip', 'VBT'],
     ['garis-garis', 'NNO'],
     ['buku', 'NNO'],
     ['?', 'PUN']]],
   'entities': []},
  {'question': 'Apa yang warga kota Monroeville sebut sebagai turis ke kota mereka?',
   'id': '56d3f1872ccc5a1400d82f77',
   'answers': [{'text': 'Kelompok Mockingbird', 'answer_start': 779}],
   'is_impossible': False,
   'indonesian_answers': [{'text': 'kelompok Mockingbird',
     'answer_start': 850,
     'answer_end': 870}],
   'postags': [[['Apa', 'PRI'],
     ['yang', 'PRR'],
     ['warga', 'NNO'],
     ['kota', 'NNP'],
     ['Monroeville', 'NNP'],
     ['seb

In [ ]:
print(f'Saving to: {SAVE_PATH}')
df_squad.to_json(SAVE_PATH)